In [36]:
# This script aims to perform rotation detection by checking the rotation, speed features etc.
# The performance is not good for this way.
# Jincheng Pang 08/26/2015
#
# The new detection algorithm based on extrem points seems to be very promising 09/14/2015 
# Optimize the code structure and convert the rotation peaks into subtitle items for more video testing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import scipy.signal as sg


# Converted from Dmitri's R code for rotation calcuation
# Added some parts for speed calcuation
def speeds_calculation(df):
    nrow,ncol = df.shape
    df['y'] = df.Angle*np.pi/180
    df['s'] = 0 
    # Large angle change between adjacent frames might be caused from +/- pi flucuation 
    for jj in xrange(1,nrow):
        df.loc[jj,'s'] = df.loc[jj-1,'s']
        diffValue = df.loc[jj,'y']-df.loc[jj-1,'y']
        diffPos = max(abs(df.loc[jj,'Cx']-df.loc[jj-1,'Cx']),abs(df.loc[jj,'Cy']-df.loc[jj-1,'Cy']))
        if np.abs(diffValue) <= 2:
            continue
        
        if diffValue > 2:
            df.loc[jj,'y'] = np.pi - df.loc[jj,'y']
        elif diffValue < -2:
            df.loc[jj,'s'] = df.loc[jj,'s'] + np.pi
            
        if diffPos > 30:
            df.loc[jj,'Cx'] = df.loc[jj-1,'Cx']
            df.loc[jj,'Cy'] = df.loc[jj-1,'Cy']
            
    # Calculate speed 
    df['vec'] = 0
    df['dist'] = 0
    
    df.loc[range(1,nrow),'vec'] = np.sqrt(np.square(np.diff(df.Cx))+np.square(np.diff(df.Cy)))
    df.loc[range(1,nrow),'dist'] = np.cumsum(df.loc[range(1,nrow),'vec'])
    
    df['rot'] = df.s + df.y
    
    return df

def generate_annotation(indNum,fs):
    indNum = indNum/fs
    hourNum = int(indNum//3600)
    indNum = indNum%3600
    minNum = int(indNum//60)
    indNum = indNum%60
    secNum = int(math.floor(indNum))
    miniSecNum = int((indNum-secNum)*100)
    return str(hourNum).zfill(2), str(minNum).zfill(2),str(secNum).zfill(2),str(miniSecNum).zfill(2)

def rotation_detection(maxtab, mintab):
    
    # re-arrange these peak points according to time points
    maxNum = len(maxtab)
    minNum = len(mintab)
    maxtabEx = np.ones((maxNum,3))
    mintabEx = np.zeros((minNum,3))
    maxtabEx[:,:-1] = maxtab
    mintabEx[:,:-1] = mintab
    
    extremNum = maxNum + minNum
    
    peakInfo = np.zeros((extremNum,3))
    
    ii = 0
    jj = 0
    kk = 0
    while ii < maxNum and jj < minNum:
        
        if maxtabEx[ii,0] < mintabEx[jj,0]:
            peakInfo[kk,:] = maxtabEx[ii,:]
            ii = ii + 1
        else:
            peakInfo[kk,:] = mintabEx[jj,:]
            jj = jj + 1
        kk = kk + 1
        
        
    if kk < extremNum:
        if ii < maxNum:
            peakInfo[kk:extremNum,:] = maxtabEx[ii:maxNum,:]
        if jj < minNum:
            peakInfo[kk:extremNum,:] = mintabEx[jj:minNum,:]
            
    #return peakInfo, maxtabEx, mintabEx
    # Work on this peakInfo array to detect rotaions
    rotPeaks = np.array([])
    for ii in xrange(extremNum-2):
        if peakInfo[ii,2] != peakInfo[ii+1,2] and  peakInfo[ii,2] ==  peakInfo[ii+2,2]:
            diff1 = peakInfo[ii,1]-peakInfo[ii+1,1]
            diff2 = peakInfo[ii+1,1]-peakInfo[ii+2,1]
            if np.absolute(diff1)>100 and np.absolute(diff2)>120: # angle difference should be large enough
                if peakInfo[ii+2,0]-peakInfo[ii,0] < 20:# happenned in short time
                    rotPeaks = np.append(rotPeaks, peakInfo[ii,:],axis = 1)
                    rotPeaks = np.append(rotPeaks, peakInfo[ii+1,:],axis = 1)
                    rotPeaks = np.append(rotPeaks, peakInfo[ii+2,:],axis = 1)
    
    rotPeaks = np.reshape(rotPeaks, newshape=(rotPeaks.shape[0]/3, 3))
    # Remove duplated entries
    frameNum = rotPeaks[:,0]
    u, indices = np.unique(frameNum, return_index=True)
    rotPeaks1 = rotPeaks[indices,:]
    
    # Convert detected rotation peaks into items for text output
    rotItems = []
    rotFrames = []
    ii = 0
    jj = 1
    while jj < len(rotPeaks1):
        startFrame = rotPeaks1[ii,0]-15
        if rotPeaks1[jj,0]-rotPeaks1[ii,0] < 30:
            jj =  jj+1
        else:
            endFrame = rotPeaks1[jj-1,0]+15
            ii = jj
            jj = jj+1
            
            hh1,mm1,ss1,ll1 = generate_annotation(startFrame,fs)
            hh2,mm2,ss2,ll2 = generate_annotation(endFrame,fs)
            startTime =  hh1 + ':' + mm1 + ':' + ss1 + ':' + ll1 
            endTime =  hh2 + ':' +  mm2 + ':' + ss2 + ':' + ll2 

            item = [startTime, endTime, 'rotation']
            rotItems.append(item)
            rotFrames.append([startFrame, endFrame])
                
    return rotPeaks1, rotItems, rotFrames
                    
                    

In [11]:
featureFileName = 'C:\\PostDoctorProjects\\VideoEEGData\\Shank2_20150609\\video_stitched\\Shan2_20150609_out_bottom.csv'
df = pd.read_csv(featureFileName)
    

In [12]:
df1 = df[df['animal']=='73-1']
df1 = df1.reset_index(drop=True)

df2 = df[df['animal']=='77-5']
df2 = df2.reset_index(drop=True)

df3 = df[df['animal']=='73-4']
df3 = df3.reset_index(drop=True)

df4 = df[df['animal']=='88-2']
df4 = df4.reset_index(drop=True)

df5 = df[df['animal']=='77-3']
df5 = df5.reset_index(drop=True)

df6 = df[df['animal']=='73-6']
df6 = df6.reset_index(drop=True)

df7 = df[df['animal']=='87-7']
df7 = df7.reset_index(drop=True)

tic = time.time()
df1 = speeds_calculation(df1)
toc = time.time()
print toc-tic
tic = time.time()
df2 = speeds_calculation(df2)
toc = time.time()
print toc-tic
tic = time.time()
df3 = speeds_calculation(df3)
toc = time.time()
print toc-tic
tic = time.time()
df4 = speeds_calculation(df4)
toc = time.time()
print toc-tic
tic = time.time()
df5 = speeds_calculation(df5)
toc = time.time()
print toc-tic
tic = time.time()
df6 = speeds_calculation(df6)
toc = time.time()
print toc-tic
tic = time.time()
df7 = speeds_calculation(df7)
toc = time.time()
print toc-tic

47.8050000668
43.9319999218
46.8109998703
45.6349999905
46.9260001183
53.0349998474
50.1050000191


In [27]:
from peakdet import peakdet
fs = 15.0
nfr = range(24000-df1.ifr[0],26000-df1.ifr[0])
delta = 30
[maxtab, mintab] = peakdet(np.array(df1.y[nfr]/np.pi*180), delta, np.array(df1.ifr[nfr]))

rottab, rotItems, rotFrames = rotation_detection(maxtab, mintab)

plt.subplot(3,3,1)
plt.plot(df1.ifr[nfr]/fs/60,df1.y[nfr]/np.pi*180)
plt.plot(maxtab[:,0]/fs/60,maxtab[:,1],'o')
plt.plot(mintab[:,0]/fs/60,mintab[:,1],'*')
plt.plot(rottab[:,0]/fs/60,rottab[:,1],'d',markersize = 12)
#plt.plot(df1.ifr[nfr]/fs/60,df1.s[nfr]/np.pi/180,'b')
#plt.plot(df1.ifr[nfr]/fs/60,df1.Angle[nfr],'k')
plt.xlabel("Minutes")
plt.ylabel("Smoothed Angles")
plt.title(df1.loc[0,'animal'])
plt.subplot(3,3,2)
plt.plot(df1.ifr[nfr]/fs/60,df1.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df1.loc[0,'animal'])
plt.title(df1.loc[0,'animal'])
plt.subplot(3,3,3)
plt.plot(df1.ifr[nfr]/fs/60,df1.Angle[nfr])
plt.xlabel("Minutes")
plt.ylabel("Raw Angles")
plt.title(df1.loc[0,'animal'])


nfr = range(29200-df1.ifr[0],30200-df1.ifr[0])
[maxtab, mintab] = peakdet(np.array(df1.y[nfr]/np.pi*180), delta, np.array(df1.ifr[nfr]))
rottab, rotItems, rotFrames = rotation_detection(maxtab, mintab)

plt.subplot(3,3,4)
plt.plot(df1.ifr[nfr]/fs/60,df1.y[nfr]/np.pi*180)
plt.plot(maxtab[:,0]/fs/60,maxtab[:,1],'o')
plt.plot(mintab[:,0]/fs/60,mintab[:,1],'*')
plt.plot(rottab[:,0]/fs/60,rottab[:,1],'d',markersize = 12)
plt.xlabel("Minutes")
plt.ylabel("Angles")
plt.title(df1.loc[0,'animal'])
plt.subplot(3,3,5)
plt.plot(df1.ifr[nfr]/fs/60,df1.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df1.loc[0,'animal'])
plt.title(df1.loc[0,'animal'])
plt.subplot(3,3,6)
plt.plot(df1.ifr[nfr]/fs/60,df1.Angle[nfr])
plt.xlabel("Minutes")
plt.ylabel("Raw Angles")
plt.title(df1.loc[0,'animal'])

nfr = range(43000-df1.ifr[0],44000-df1.ifr[0])
[maxtab, mintab] = peakdet(np.array(df1.y[nfr]/np.pi*180), delta, np.array(df1.ifr[nfr]))
rottab, rotItems, rotFrames = rotation_detection(maxtab, mintab)

plt.subplot(3,3,7)
plt.plot(df1.ifr[nfr]/fs/60,df1.y[nfr]/np.pi*180)
plt.plot(maxtab[:,0]/fs/60,maxtab[:,1],'o')
plt.plot(mintab[:,0]/fs/60,mintab[:,1],'*')
plt.plot(rottab[:,0]/fs/60,rottab[:,1],'d',markersize = 12)
plt.xlabel("Minutes")
plt.ylabel("Angles")
plt.title(df1.loc[0,'animal'])
plt.subplot(3,3,8)
plt.plot(df1.ifr[nfr]/fs/60,df1.rot[nfr]/2/np.pi)
plt.xlabel("Minutes")
plt.ylabel("Rotation")
plt.title(df1.loc[0,'animal'])
plt.title(df1.loc[0,'animal'])
plt.subplot(3,3,9)
plt.plot(df1.ifr[nfr]/fs/60,df1.Angle[nfr])
plt.xlabel("Minutes")
plt.ylabel("Raw Angles")
plt.title(df1.loc[0,'animal'])
plt.show()


In [37]:
nfr = df1.ifr-df1.ifr[0]
delta = 30
[maxtab, mintab] = peakdet(np.array(df1.y[nfr]/np.pi*180), delta, np.array(df1.ifr[nfr]))

rottab, rotItems, rotFrames = rotation_detection(maxtab, mintab)


for item in rotItems:
    print item[0], item[1], item[2]


00:19:28:53 00:19:31:33 rotation
00:19:54:06 00:19:56:66 rotation
00:20:55:66 00:20:58:59 rotation
00:21:52:53 00:21:55:79 rotation
00:23:05:20 00:23:08:46 rotation
00:25:53:66 00:25:56:93 rotation
00:27:00:73 00:27:03:53 rotation
00:27:32:53 00:27:35:73 rotation
00:28:09:20 00:28:12:40 rotation
00:30:21:00 00:30:23:53 rotation
00:32:52:26 00:32:55:53 rotation
00:35:16:80 00:35:20:06 rotation
00:36:17:59 00:36:20:73 rotation
00:37:17:86 00:37:20:66 rotation
00:38:47:93 00:38:50:06 rotation
00:38:53:53 00:38:56:80 rotation
00:42:16:19 00:42:19:66 rotation
00:42:57:46 00:42:59:80 rotation
00:44:35:53 00:44:39:06 rotation
00:45:13:46 00:45:16:53 rotation
00:46:45:19 00:46:48:33 rotation
00:46:49:33 00:46:52:26 rotation
00:46:55:66 00:46:58:86 rotation
00:46:59:80 00:47:02:86 rotation


In [38]:
nfr = df2.ifr-df2.ifr[0]
delta = 30
[maxtab, mintab] = peakdet(np.array(df2.y[nfr]/np.pi*180), delta, np.array(df2.ifr[nfr]))

rottab, rotItems, rotFrames = rotation_detection(maxtab, mintab)


for item in rotItems:
    print item[0], item[1], item[2]

01:00:36:26 01:00:39:00 rotation
01:04:10:06 01:04:13:26 rotation
01:05:38:06 01:05:41:00 rotation
01:06:32:73 01:06:35:26 rotation
01:09:28:46 01:09:31:66 rotation
01:09:38:66 01:09:41:60 rotation


In [39]:
nfr = df3.ifr-df3.ifr[0]
delta = 30
[maxtab, mintab] = peakdet(np.array(df3.y[nfr]/np.pi*180), delta, np.array(df3.ifr[nfr]))

rottab, rotItems, rotFrames = rotation_detection(maxtab, mintab)


for item in rotItems:
    print item[0], item[1], item[2]

01:29:44:13 01:29:46:73 rotation
01:30:28:86 01:30:31:80 rotation
01:31:07:66 01:31:10:73 rotation
01:31:38:60 01:31:41:86 rotation
01:32:12:93 01:32:15:86 rotation
01:32:14:93 01:32:18:19 rotation
01:34:03:66 01:34:05:86 rotation
01:34:23:39 01:34:26:53 rotation
01:35:38:66 01:35:41:66 rotation
01:35:43:39 01:35:47:13 rotation
01:35:45:60 01:35:48:39 rotation
01:37:58:93 01:38:02:06 rotation
01:38:01:66 01:38:05:00 rotation
01:38:17:06 01:38:20:19 rotation
01:38:27:19 01:38:30:39 rotation
01:38:30:60 01:38:33:86 rotation
01:38:35:19 01:38:38:46 rotation
01:39:16:60 01:39:18:93 rotation
01:39:23:73 01:39:27:00 rotation
01:39:29:93 01:39:33:86 rotation
01:39:32:06 01:39:34:06 rotation
01:39:53:93 01:39:57:19 rotation
01:39:56:00 01:39:58:06 rotation
01:40:37:66 01:40:41:53 rotation
01:40:50:46 01:40:54:26 rotation
01:40:52:46 01:40:55:53 rotation
01:41:25:60 01:41:28:33 rotation
01:41:51:00 01:41:53:39 rotation
01:41:58:13 01:42:01:13 rotation
01:42:31:66 01:42:34:80 rotation
01:43:32:3